In [1]:
from h5io_browser import Pointer
from h5io_browser.base import _open_hdf, _read_hdf
import h5py
import posixpath
import os

In [2]:
def list_groups_and_nodes(hdf, h5_path):
    """
    Get the list of groups and list of nodes from an open HDF5 file

    Args:
        hdf (h5py.File): file handle of an open HDF5 file
        h5_path (str): path inside the HDF5 file

    Returns:
        list, list: list of groups and list of nodes
    """
    groups = set()
    nodes = set()
    try:
        h = hdf[h5_path]
        for k in h.keys():
            if isinstance(h[k], h5py.Group):
                group_attrs_dict = h[k].attrs
                if 'TITLE' in group_attrs_dict.keys() and group_attrs_dict['TITLE'] == 'multiarray':
                    nodes.add(k)
                else:
                    groups.add(k)
            else:
                nodes.add(k)
    except KeyError:
        pass
    return list(groups), list(nodes)

In [3]:
def get_h5_path(h5_path, name):
    """
    Combine the current h5_path with the relative path

    Args:
        h5_path (str): absolute path of the node in the hdf5 file
        name (str): relative path to be added to the absolute path

    Returns:
        str: combined path
    """
    return posixpath.join(h5_path, name)

In [4]:
def read_dict_from_hdf(
    file_name, h5_path, recursive=False, group_paths=[], slash="ignore"
):
    """
    Read data from HDF5 file into a dictionary - by default only the nodes are converted to dictionaries, additional
    sub groups can be specified using the group_paths parameter.
    Args:
       file_name (str): Name of the file on disk
       h5_path (str): Path to a group in the HDF5 file from where the data is read
       recursive (bool): Load all subgroups recursively
       group_paths (list): list of additional groups to be included in the dictionary, for example:
                           ["input", "output", "output/generic"]
                           These groups are defined relative to the h5_path.
       slash (str): 'ignore' | 'replace' Whether to replace the string {FWDSLASH} with the value /. This does
                    not apply to the top level name (title). If 'ignore', nothing will be replaced.
    Returns:
       dict:     The loaded data. Can be of any type supported by ``write_hdf5``.
    """

    def get_dict_from_nodes(store, h5_path, slash="ignore"):
        """
        Load all nodes from an HDF5 path into a dictionary
        Args:
            store (str): Name of the file on disk, or file-like object.  Note: for files created with the 'core'
                         driver, HDF5 still requires this be non-empty.:
            h5_path (str): Path to a group in the HDF5 file from where the data is read
            slash (str): 'ignore' | 'replace' Whether to replace the string {FWDSLASH} with the value /. This does
                         not apply to the top level name (title). If 'ignore', nothing will be replaced.
        Returns:
            dict:        The loaded data. Can be of any type supported by ``write_hdf5``.
        """
        return {
            n: _read_hdf(
                hdf_filehandle=store,
                h5_path=get_h5_path(h5_path=h5_path, name=n),
                slash=slash,
            )
            for n in list_groups_and_nodes(hdf=store, h5_path=h5_path)[1]
        }

    def resolve_nested_dict(group_path, data_dict):
        """
        Turns a dict with a key containing slashes into a nested dict.  {'/a/b/c': 1} -> {'a': {'b': {'c': 1}
        Args:
            group_path (str): path inside the HDF5 file the data_dictionary was loaded from
            data_dict (dict): dictionary with data loaded from the HDF5 file
        Returns:
            dict: hierarchical dictionary
        """
        groups = group_path.split("/")
        nested_dict = data_dict
        for g in groups[::-1]:
            nested_dict = {g: nested_dict}
        return nested_dict

    def get_groups_hdf(hdf, h5_path):
        """
        Get all sub-groups of a given HDF5 path
        Args:
            hdf (str): Name of the file on disk, or file-like object.  Note: for files created with the 'core'
                       driver, HDF5 still requires this be non-empty.:
            h5_path (str): Path to a group in the HDF5 file from where the data is read
        Returns:
            list: list of HDF5 groups
        """
        try:
            h = hdf[h5_path]
            group_lst = []
            for group in [h[k].name for k in h.keys() if isinstance(h[k], h5py.Group)]:
                group_attrs_dict = h[group].attrs
                if 'TITLE' in group_attrs_dict.keys() and group_attrs_dict['TITLE'] != 'multiarray':
                    group_lst += [group] + get_groups_hdf(hdf=hdf, h5_path=group)
                elif 'TITLE' not in group_attrs_dict.keys():
                    group_lst += [group] + get_groups_hdf(hdf=hdf, h5_path=group)
            return group_lst
        except KeyError:
            return []

    def merge_dict(main_dict, add_dict):
        """
        Merge two dictionaries recursively

        Args:
            main_dict (dict): The primary dictionary, the secondary dictionary is merged into
            add_dict (dict): The secondary dictionary which is merged in the primary dictionary

        Returns:
            dict: The merged dictionary with all keys
        """
        for k, v in add_dict.items():
            if k in main_dict.keys() and isinstance(v, dict):
                main_dict[k] = merge_dict(main_dict=main_dict[k], add_dict=v)
            else:
                main_dict[k] = v
        return main_dict

    if recursive and len(group_paths) > 0:
        raise ValueError(
            "Loading subgroups can either be defined by the group paths ",
            group_paths,
            " or by the recursive ",
            recursive,
            " parameter. Specifying both lead to this ValueError.",
        )

    with _open_hdf(file_name, mode="r") as store:
        output_dict = get_dict_from_nodes(store=store, h5_path=h5_path, slash=slash)
        if h5_path == "/" and recursive:
            group_paths = [g[1:] for g in get_groups_hdf(hdf=store, h5_path=h5_path)]
        elif h5_path[0] != "/" and recursive:
            group_paths = [
                g[len("/" + h5_path) + 1 :]
                for g in get_groups_hdf(hdf=store, h5_path="/" + h5_path)
            ]
        elif recursive:
            group_paths = [
                g[len(h5_path) + 1 :]
                for g in get_groups_hdf(hdf=store, h5_path=h5_path)
            ]
        for group_path in group_paths:
            output_dict = merge_dict(
                main_dict=output_dict,
                add_dict=resolve_nested_dict(
                    group_path=group_path,
                    data_dict=get_dict_from_nodes(
                        store=store,
                        h5_path=get_h5_path(h5_path=h5_path, name=group_path),
                        slash=slash,
                    ),
                ),
            )
    return output_dict

# Create SPHInX job

In [5]:
from pyiron_atomistics import Project

In [6]:
pr = Project("test")

In [7]:
pr.remove_jobs(recursive=True, silently=True)

  0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
structure = pr.create.structure.ase.bulk("Al", cubic=True)

In [9]:
job = pr.create.job.Sphinx("sx")

In [10]:
job.structure = structure

In [11]:
job.calc_minimize()

In [12]:
job.run()

The job sx was saved and received the ID: 1


# Read job dictionary

In [13]:
job_dict = read_dict_from_hdf(
    file_name=job.project_hdf5.file_name,
    h5_path="/",
    recursive=True,
    group_paths=[],
    slash='ignore',
)

# Get SPHInX schema

In [14]:
file_name = job.project_hdf5.file_name
file_name

'/home/jovyan/test/sx.h5'

In [15]:
key_dict = {}
def collect_attrs(name, obj):
    if len(obj.attrs) != 0:
        key_dict[name] = obj.attrs["TITLE"]
    
with h5py.File(file_name, 'r') as f:
    f.visititems(collect_attrs)

In [16]:
group_lst, node_lst = [], []
for k, v in key_dict.items():
    if len([g for g in group_lst if g in k]) == 0:
        node_lst.append(k)
        if v == "multiarray":
            group_lst.append(k)
node_lst

['sx/DICT_VERSION',
 'sx/HDF_VERSION',
 'sx/NAME',
 'sx/OBJECT',
 'sx/TYPE',
 'sx/VERSION',
 'sx/executable',
 'sx/input/generic/DICT_VERSION',
 'sx/input/generic/NAME',
 'sx/input/generic/OBJECT',
 'sx/input/generic/TYPE',
 'sx/input/generic/VERSION',
 'sx/input/generic/data_dict',
 'sx/input/generic_dict',
 'sx/input/interactive',
 'sx/input/parameters/CheckOverlap__index_18',
 'sx/input/parameters/Ediff__index_10',
 'sx/input/parameters/EmptyStates__index_4',
 'sx/input/parameters/EnCut__index_1',
 'sx/input/parameters/Estep__index_9',
 'sx/input/parameters/HDF_VERSION',
 'sx/input/parameters/Istep__index_21',
 'sx/input/parameters/KJxc__index_12',
 'sx/input/parameters/KpointCoords__index_2',
 'sx/input/parameters/KpointFolding__index_3',
 'sx/input/parameters/MethfesselPaxton__index_5',
 'sx/input/parameters/NAME',
 'sx/input/parameters/OBJECT',
 'sx/input/parameters/READ_ONLY',
 'sx/input/parameters/SaveMemory__index_13',
 'sx/input/parameters/Sigma__index_6',
 'sx/input/paramete

# Write SPHInX job

In [17]:
def get_node_from_job_dict(job_dict, node):
    node_name_lst = node.split("/")
    tmp_dict = job_dict
    for group in node_name_lst:
        tmp_dict = tmp_dict[group]
    return tmp_dict

In [18]:
node_dict = {node: get_node_from_job_dict(job_dict=job_dict, node=node) for node in node_lst}

In [19]:
os.remove(file_name)

In [20]:
with Pointer(file_name=file_name) as hdf_file:
    hdf_file.write_dict(node_dict)

# Reload job

In [21]:
job = pr.load(job.job_name)

/srv/conda/envs/notebook/lib/python3.10/site-packages/pyiron_base/interfaces/lockable.py:58: LockedWarning: __setitem__ called on <class 'pyiron_atomistics.sphinx.input_writer.Group'>, but object is locked!
  warnings.warn(


In [22]:
job.input

Group({'sphinx': Group({'pawPot': Group({'species': Group([Group({'name': '"Al"', 'potType': '"AtomPAW"', 'element': '"Al"', 'potential': '"Al_GGA.atomicdata"'})])}), 'structure': Group({'cell': array([[7.6533908, 0.       , 0.       ],
       [0.       , 7.6533908, 0.       ],
       [0.       , 0.       , 7.6533908]]), 'species': Group([Group({'element': '"Al"', 'atom': Group([Group({'label': '"spin_0.0"', 'coords': array([0., 0., 0.]), 'movable': 1}), Group({'label': '"spin_0.0"', 'coords': array([0.       , 3.8266954, 3.8266954]), 'movable': 1}), Group({'label': '"spin_0.0"', 'coords': array([3.8266954, 0.       , 3.8266954]), 'movable': 1}), Group({'label': '"spin_0.0"', 'coords': array([3.8266954, 3.8266954, 0.       ]), 'movable': 1})])})])}), 'basis': Group({'eCut': 24.989539079445393, 'kPoint': Group({'coords': array([0.5, 0.5, 0.5]), 'weight': 1, 'relative': 1}), 'folding': array([4, 4, 4]), 'saveMemory': 1}), 'PAWHamiltonian': Group({'nEmptyStates': 7, 'ekt': 0.2, 'MethfesselPaxton': 1, 'xc': 'PBE', 'spinPolarized': 0}), 'initialGuess': Group({'waves': Group({'pawBasis': 1, 'lcao': Group([])}), 'rho': Group({'atomicOrbitals': 1}), 'noWavesStorage': 0}), 'main': Group({'ricQN': Group({'maxSteps': '100', 'maxStepLength': '0.18897261246257704', 'bornOppenheimer': Group({'scfDiag': Group({'rhoMixing': '1.0', 'spinMixing': '1.0', 'dEnergy': 3.674932217565499e-06, 'maxSteps': '100', 'preconditioner': Group({'type': 'KERKER', 'scaling': 1.0, 'spinScaling': 1.0}), 'blockCCG': Group([])})})})})}), 'EnCut': 340, 'KpointCoords': [0.5, 0.5, 0.5], 'KpointFolding': [4, 4, 4], 'EmptyStates': 7, 'MethfesselPaxton': 1, 'Sigma': 0.2, 'Xcorr': 'PBE', 'VaspPot': 0, 'Estep': 100, 'Ediff': 0.0001, 'WriteWaves': 1, 'KJxc': 0, 'SaveMemory': 1, 'rhoMixing': 1.0, 'spinMixing': 1.0, 'rhoResidualScaling': 1.0, 'spinResidualScaling': 1.0, 'CheckOverlap': 1, 'THREADS': 1, 'use_on_the_fly_cg_optimization': 1, 'Istep': 100})

In [23]:
job.output.generic

DataContainer({'dft': DataContainer({'energy_free': array([-228.78316018, -228.78315944]), 'n_valence': {'Al': 3}, 'bands_k_weights': array([0.125, 0.375, 0.375, 0.125]), 'kpoints_cartesian': array([[0.1592067, 0.1592067, 0.1592067],
       [0.1592067, 0.1592067, 0.4776201],
       [0.1592067, 0.4776201, 0.4776201],
       [0.4776201, 0.4776201, 0.4776201]]), 'bands_e_fermi': array([5.864736, 5.702807, 5.702836, 5.702855]), 'bands_occ': array([[[ 2.    ,  2.    ,  2.    ,  2.    , -0.0703, -0.0704, -0.0704,
          0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
        [ 2.    ,  2.    ,  2.    ,  2.    ,  2.    ,  2.    , -0.    ,
         -0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
        [ 2.    ,  2.    ,  2.    ,  2.    ,  2.    ,  1.0356,  1.0347,
          0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
        [ 2.    ,  2.    ,  2.    ,  2.    ,  2.    ,  2.    ,  2.    ,
          2.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ]],

       [[ 2.    ,  2.    ,  2.    ,  2.    , -0.0704, -0.0704, -0.0704,
          0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
        [ 2.    ,  2.    ,  2.    ,  2.    ,  2.    ,  2.    , -0.    ,
         -0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
        [ 2.    ,  2.    ,  2.    ,  2.    ,  2.    ,  1.0352,  1.0352,
          0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
        [ 2.    ,  2.    ,  2.    ,  2.    ,  2.    ,  2.    ,  2.    ,
          2.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ]]]), 'bands_eigen_values': array([[[-5.14  ,  1.4238,  1.4239,  1.4239,  5.9394,  5.9395,  5.9395,
          7.7794,  7.7795,  7.7795, 11.4633, 11.4633, 11.4635],
        [-4.0193, -1.8032,  2.4931,  2.4931,  4.6313,  4.6314,  7.0212,
          7.0213,  8.8035,  9.1646,  9.1649, 10.894 , 11.1662],
        [-2.9064, -0.7059, -0.7059,  1.4615,  3.5798,  5.6986,  5.6987,
          7.8085,  8.0932, 10.1434, 10.1435, 12.2068, 13.4422],
        [-1.8011,  0.3824,  0.3824,  0.3824,  2.5573,  2.5575,  2.5575,
          4.693 , 13.1792, 13.1793, 13.1793, 14.4332, 14.4334]],

       [[-5.14  ,  1.4239,  1.4239,  1.4239,  5.9395,  5.9395,  5.9395,
          7.7795,  7.7795,  7.7795, 11.4635, 11.4635, 11.4635],
        [-4.0192, -1.8032,  2.4931,  2.4931,  4.6314,  4.6314,  7.0213,
          7.0213,  8.8036,  9.1648,  9.1648, 10.8941, 11.1662],
        [-2.9064, -0.7059, -0.7059,  1.4616,  3.5798,  5.6987,  5.6987,
          7.8086,  8.0932, 10.1435, 10.1435, 12.2069, 13.4422],
        [-1.8011,  0.3824,  0.3824,  0.3824,  2.5575,  2.5575,  2.5575,
          4.6931, 13.1794, 13.1794, 13.1794, 14.4334, 14.4334]]]), 'scf_convergence': [True, True], 'scf_energy_int': array([array([-227.76073135, -228.76487746, -228.75055037, -228.75030395,
              -228.75029236])                                            ,
       array([-228.75029155, -228.75029152])], dtype=object), 'scf_energy_free': array([array([-227.79757673, -228.79771858, -228.78341073, -228.78317077,
              -228.78316018])                                            ,
       array([-228.78315943, -228.78315944])], dtype=object), 'scf_computation_time': array([array([1.292255, 2.575304, 3.85743 , 5.171744, 6.476227]),
       array([7.729495, 9.018226])], dtype=object), 'scf_energy_zero': array([array([-227.78836538, -228.7895083 , -228.77519564, -228.77495407,
              -228.77494323])                                            ,
       array([-228.77494246, -228.77494246])], dtype=object), 'scf_energy_band': array([array([15.05566482, 13.62260753, 13.80978426, 13.83579452, 13.84051221]),
       array([13.84081386, 13.84101605])], dtype=object), 'scf_electronic_entropy': array([array([5.01306946, 4.46826186, 4.47087874, 4.47175913, 4.47189425]),
       array([4.47190311, 4.4719075 ])], dtype=object), 'scf_residue': array([array([5.57072e-03, 1.65550e-03, 2.37401e-04, 4.03935e-05, 6.62098e-06]),
       array([1.48271e-06, 1.86194e-07])], dtype=object), 'energy_int': arr